In [9]:
import pandas as pd
import geojson

ModuleNotFoundError: No module named 'geojson'

In [2]:
DATA_PATH = "./data/mexico2010.csv"
BORDER_STATIONS_PATH = "./data/border_stations7.geojson"

In [6]:
with open(BORDER_STATIONS_PATH) as bs:
    border_stations = geojson.load(bs)

feature_df = json_normalize(border_stations["features"])


NameError: name 'geojson' is not defined

In [8]:
!pip3 install geojson


  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
